<a href="https://colab.research.google.com/github/tomfox1/DS-Unit-2-Sprint-3-Classification-Validation/blob/master/Copy_of_DS_Unit_2_Sprint_Challenge_3_Classification_Validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 _Lambda School Data Science Unit 2_
 
 # Classification & Validation Sprint Challenge

Follow the instructions for each numbered part to earn a score of 2. See the bottom of the notebook for a list of ways you can earn a score of 3.

#### For this Sprint Challenge, you'll predict whether a person's income exceeds $50k/yr, based on census data.

You can read more about the Adult Census Income dataset at the UCI Machine Learning Repository: https://archive.ics.uci.edu/ml/datasets/adult

#### Run this cell to load the data:

In [1]:
!pip install category_encoders

     |████████████████████████████████| 92kB 5.2MB/s 


In [0]:
import category_encoders as ce
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score

In [3]:
import pandas as pd

columns = ['age', 
           'workclass', 
           'fnlwgt', 
           'education', 
           'education-num', 
           'marital-status', 
           'occupation', 
           'relationship', 
           'race', 
           'sex', 
           'capital-gain', 
           'capital-loss', 
           'hours-per-week', 
           'native-country', 
           'income']

df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data', 
                 header=None, names=columns)

df['income'] = df['income'].str.strip()
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


## Part 1 — Begin with baselines

Split the data into an **X matrix** (all the features) and **y vector** (the target).

(You _don't_ need to split the data into train and test sets here. You'll be asked to do that at the _end_ of Part 1.)

In [0]:
target = ['income']

X = df[columns].drop(columns='income')
y = df[target]

What **accuracy score** would you get here with a **"majority class baseline"?** 
 
(You can answer this question either with a scikit-learn function or with a pandas function.)

In [5]:
#With a "majority class baseline" we'd have an accuracy score of 75.92%
#in other words, by predicting the majority class "<=50k" 100% of the time we'd be right 75.92% of the time
#our model will have to have a higher accuracy score then the naive baseline model in order to compensate for further model iterations/tweaks
df['income'].value_counts(normalize=True)

<=50K    0.75919
>50K     0.24081
Name: income, dtype: float64

What **ROC AUC score** would you get here with a **majority class baseline?**

(You can answer this question either with a scikit-learn function or with no code, just your understanding of ROC AUC.)

In [0]:
#Our ROC AUC score would be of 0.5
#our area would lie under the diagonal x=y at 45 degrees  

In this Sprint Challenge, you will use **"Cross-Validation with Independent Test Set"** for your model validaton method.

First, **split the data into `X_train, X_test, y_train, y_test`**. You can include 80% of the data in the train set, and hold out 20% for the test set.

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((26048, 14), (6513, 14), (26048, 1), (6513, 1))

## Part 2 — Modeling with Logistic Regression!

- You may do exploratory data analysis and visualization, but it is not required.
- You may **use all the features, or select any features** of your choice, as long as you select at least one numeric feature and one categorical feature.
- **Scale your numeric features**, using any scikit-learn [Scaler](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.preprocessing) of your choice.
- **Encode your categorical features**. You may use any encoding (One-Hot, Ordinal, etc) and any library (category_encoders, scikit-learn, pandas, etc) of your choice.
- You may choose to use a pipeline, but it is not required.
- Use a **Logistic Regression** model.
- Use scikit-learn's [**cross_val_score**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html) function. For [scoring](https://scikit-learn.org/stable/modules/model_evaluation.html#the-scoring-parameter-defining-model-evaluation-rules), use **accuracy**.
- **Print your model's cross-validation accuracy score.**

In [8]:
#the preprocessor function will make it easier to encode, impute and scale our features 

features = ['age', 
           'workclass', 
           'fnlwgt', 
           'education', 
           'education-num', 
           'marital-status', 
           'occupation', 
           'relationship', 
           'race', 
           'sex', 
           'capital-gain', 
           'capital-loss', 
           'hours-per-week', 
           'native-country']

target = 'income'

preprocessor = make_pipeline(ce.OrdinalEncoder(), SimpleImputer(), MinMaxScaler())
X = preprocessor.fit_transform(X_train[features])
X = pd.DataFrame(X, columns=features)
y = y_train[target]
X.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,0.219178,0.000,0.126253,0.000000,0.800000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.500000,0.0
1,0.260274,0.125,0.050376,0.066667,0.666667,0.166667,0.071429,0.2,0.0,1.0,0.0,0.433196,0.500000,0.0
2,0.561644,0.250,0.129551,0.133333,0.266667,0.333333,0.142857,0.0,0.0,1.0,0.0,0.000000,0.397959,0.0
3,0.054795,0.125,0.114033,0.066667,0.666667,0.166667,0.214286,0.2,0.0,1.0,0.0,0.000000,0.459184,0.0
4,0.136986,0.125,0.181733,0.200000,0.600000,0.500000,0.285714,0.0,0.0,1.0,0.0,0.000000,0.397959,0.0


In [16]:
model = LogisticRegression(solver='lbfgs', max_iter=1000)
model.fit(X, y)

cross_val_score(model, X, y, scoring='accuracy', cv=10).mean()

0.8211381721904291

## Part 3 — Modeling with Tree Ensembles!

Part 3 is the same as Part 2, except this time, use a **Random Forest** or **Gradient Boosting** classifier. You may use scikit-learn, xgboost, or any other library. Then, print your model's cross-validation accuracy score.

In [18]:
#our cross validation scores are slightly better in our random forest model compared to our logistic regression model 

model = RandomForestClassifier(max_depth=4, n_estimators=100, n_jobs=-1, random_state=42)
model.fit(X, y)

cross_val_score(model, X, y, scoring="accuracy", cv=10).mean()

0.8422146056119185

In [19]:
#gradient boosting yields the highest score for our model 

model = XGBClassifier(max_depth=3, n_estimators=100, n_jobs=-1, random_state=42)
model.fit(X, y)

cross_val_score(model, X, y, scoring="accuracy", cv=10).mean()

0.8621395402319184

## Part 4 — Calculate classification metrics from a confusion matrix

Suppose this is the confusion matrix for your binary classification model:

<table>
  <tr>
    <td colspan="2" rowspan="2"></td>
    <td colspan="2">Predicted</td>
  </tr>
  <tr>
    <td>Negative</td>
    <td>Positive</td>
  </tr>
  <tr>
    <td rowspan="2">Actual</td>
    <td>Negative</td>
    <td style="border: solid">85</td>
    <td style="border: solid">58</td>
  </tr>
  <tr>
    <td>Positive</td>
    <td style="border: solid">8</td>
    <td style="border: solid"> 36</td>
  </tr>
</table>

Calculate accuracy

In [12]:
true_positives=36
true_negatives = 85 
false_positives = 58
false_negatives = 8 
total = true_positives + true_negatives + false_positives + false_negatives

accuracy = (true_positives + true_negatives)/total 
print(accuracy)


0.6470588235294118


Calculate precision

In [13]:
precision = true_positives / (true_positives + false_positives)
precision

0.3829787234042553

Calculate recall

In [14]:
recall = true_positives / (true_positives + false_negatives)
recall

0.8181818181818182

## BONUS — How you can earn a score of 3

### Part 1
Do feature engineering, to try improving your cross-validation score.

### Part 2
Experiment with feature selection, preprocessing, categorical encoding, and hyperparameter optimization, to try improving your cross-validation score.

### Part 3
Which model had the best cross-validation score? Refit this model on the train set and do a final evaluation on the held out test set — what is the test score? 

### Part 4
Calculate F1 score and False Positive Rate. 